In [1]:
!pip install torch torchvision tensorflow numpy matplotlib

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.6 MB 840.2 kB/s eta 0:00:02
   -------------------- ------------------- 0.8/1.6 MB 960.2 kB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.6 MB 915.5 kB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.6 MB 915.5 kB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.8/204.2 MB 1.2 MB/s eta 0:02:50
   ---------------------------------------- 0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandasai 2.4.2 requires torch==2.4.1; sys_platform != "darwin", but you have torch 2.6.0 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
image_size = 64
batch_size = 128
latent_dim = 100
epochs = 5000
learning_rate = 0.0002

In [ ]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize between -1 and 1
])


In [ ]:
dataset = ImageFolder(root="images", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, image_size * image_size * 3),
            nn.Tanh()  # Output between -1 and 1
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], 3, image_size, image_size)
        return img


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(image_size * image_size * 3, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )


In [ ]:
def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity


In [ ]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)


In [ ]:
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))


In [ ]:
for epoch in range(epochs):
    for batch in dataloader:
        real_images, _ = batch
        real_images = real_images.to(device)

        # Generate fake images
        z = torch.randn(real_images.shape[0], latent_dim).to(device)
        fake_images = generator(z)

        # Train Discriminator
        real_labels = torch.ones(real_images.shape[0], 1).to(device)
        fake_labels = torch.zeros(real_images.shape[0], 1).to(device)

        optimizer_D.zero_grad()
        real_loss = criterion(discriminator(real_images), real_labels)
        fake_loss = criterion(discriminator(fake_images.detach()), fake_labels)
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        fake_output = discriminator(fake_images)
        g_loss = criterion(fake_output, real_labels)
        g_loss.backward()
        optimizer_G.step()

In [ ]:
if epoch % 500 == 0:
        save_image(fake_images[:25], f"generated_images/epoch_{epoch}.png", nrow=5, normalize=True)
        print(f"Epoch [{epoch}/{epochs}] | D Loss: {d_loss.item()} | G Loss: {g_loss.item()}")

# Save Model
torch.save(generator.state_dict(), "cat_dog_generator.pth")
